<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/03-BERT-beto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo a evaluar JCC: BERT

En esta notebook se presentan los experimentos sobre la estrategia de representación y técnica de aprendizaje *avanzada* utilizada para las JCC de  la Universidad Nacional de La Plata.

Para ello vamos a tomar el texto de la consulta de los correos y aplicar BERT con el modelo pre-entrenado en español (BETO - Departamento de Computación de la Universidad de Chile).


## Instalación de librerías

Se instalan dos librerías que no están en el entorno Colab:
- *simpletransformers* (para entrenar Bert),
- requests (para consumir funciones propias de Github),
- wget (para la descarga de archivos).

In [1]:
!pip install simpletransformers
!pip install requests
!pip install wget

### Funciones útiles

Se cargan funciones útiles desde el repo https://github.com/jumafernandez/clasificacion_correos para la carga y balanceo del dataset.

In [2]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/jcc/funciones_dataset.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_dataset.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_dataset import get_clases, cargar_dataset, separar_x_y_rna

También se carga la función para preprocesar el texto que se usó en los otros modelos desde el repo: https://github.com/jumafernandez/clasificacion_correos.

In [3]:

import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/jcc/funciones_preprocesamiento.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_preprocesamiento.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_preprocesamiento import preprocesar_correos_bert

## Carga del dataset con los correos

Se cargan los datos y se realiza el balanceo de clases:

In [4]:
from funciones_dataset import get_clases, cargar_dataset
from os import path
import warnings
warnings.filterwarnings("ignore")

# Cantidad de clases
CANTIDAD_CLASES = 4

# Constantes con los datos
DS_DIR = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/consolidado_jcc/'
TRAIN_FILE = 'correos-train-80.csv'
TEST_FILE = 'correos-test-20.csv'

# Chequeo sobre si los archivos están en el working directory
download_files = not(path.exists(TRAIN_FILE))

etiquetas = get_clases()
train_df, test_df, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')



El conjunto de entrenamiento tiene la dimensión: (800, 24)
El conjunto de testeo tiene la dimensión: (200, 24)


### Pre-procesamiento de los datos

Se preparan los datos para el entrenamiento de BERT:

In [5]:
train_df = train_df[['Consulta', 'clase']]
train_df.columns = ['text', 'labels']
test_df = test_df[['Consulta', 'clase']]
test_df.columns = ['text', 'labels']

# Cambio los integers por las etiquetas
train_df.labels = etiquetas[train_df.labels]
test_df.labels = etiquetas[test_df.labels]

# Las vuelvo a pasar a números 0-N para evitar conflictos con simpletransformers
# Este paso está fijo para estos experimentos
dict_clases_id = {'Otras Consultas': 0,
                            'Ingreso a la Universidad': 1,
                            'Boleto Universitario': 2,
                            'Requisitos de Ingreso': 3}

train_df['labels'].replace(dict_clases_id, inplace=True)
test_df['labels'].replace(dict_clases_id, inplace=True)

# Muestro salida por consola
print('Existen {} clases: {}.'.format(len(train_df.labels.unique()), train_df.labels.unique()))

Existen 4 clases: [0 1 2 3].


## Elección de un modelo monolenguaje pre-entrenado

La librería *simpletransformers* se basa en la librería *Transformers* de HuggingFace. Esto permite utilizar todos los modelos pre-entrenados disponibles en la [Transformers library](https://huggingface.co/transformers/pretrained_models.html) que son provistos por toda la comunidad de desarrolladores. Para ver cuales son los modelos disponibles, se puede ingresar a [https://huggingface.co/models](https://huggingface.co/models).

En nuestro caso, vamos a utilizar el modelo `dccuchile/bert-base-spanish-wwm-cased`. Este modelo está pre-entrenado por un equipo de investigadores del Departamento de Computación de la Universidad de Chile.

### Definición del Modelo para Clasificación

Se carga el modelo pre-entrenado BETO con la respectiva definición de hiperparámetros para el entrenamiento:

In [6]:
from simpletransformers.classification import ClassificationModel

# Cantidad de epochs
epocas = 4

# Hiperparámetros
train_args = {
        'overwrite_output_dir': True,
        'num_train_epochs': epocas,
        'fp16': True,
        'learning_rate': 4e-5,
        'do_lower_case': True,
        'use_early_stopping': True,
        }

# Creamos el ClassificationModel
model = ClassificationModel(
    model_type='bert', 
#    model_name='bert-base-multilingual-cased',
    model_name='dccuchile/bert-base-spanish-wwm-cased',
    num_labels=CANTIDAD_CLASES,
    use_cuda=False,
    args=train_args
)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Se verifican los parámetros del modelo:

In [7]:
model.args

ClassificationArgs(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=True, adafactor_scale_parameter=True, adafactor_warmup_init=True, adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=True, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=False, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=128, model_name

## Entrenamiento

Se entrena el modelo con el dataset de train en función de los hiperparámetros:

In [8]:
# Entrenamos el modelo
model.train_model(train_df)

  0%|          | 0/800 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/100 [00:00<?, ?it/s]

(400, 0.3883092541916994)

## Testeo del modelo entrenado

Se testea el modelo sobre el 20% de instancias reservadas para esta tarea:

In [9]:
# Evaluamos el modelo
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Ejecutamos las predicciones sobre testing
predictions, raw_outputs = model.predict(list(test_df.text))

# Calculo las métricas sobre test para el paper
acc_test = accuracy_score(test_df.labels, predictions)
precision_test = precision_score(test_df.labels, predictions, average='macro')
recall_test = recall_score(test_df.labels, predictions, average='macro')
f1_test = f1_score(test_df.labels, predictions, average='macro')

# Genero un diccionario con los parámetro y el acc en test
dict_test = {}
dict_test['clasificador'] = 'BETO'
dict_test['accuracy'] = acc_test
dict_test['precision'] = precision_test
dict_test['recall'] = recall_test
dict_test['f1_score'] = f1_test
 
# Paso el diccionario a dataframe y lo guardo en un archivo con fecha/hora
results_test = pd.DataFrame([dict_test])
print(results_test)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  clasificador  accuracy  precision   recall  f1_score
0         BETO     0.885   0.861761  0.86699  0.864081


Se guardan los resultados en un archivo csv:

In [10]:
# Lo guardo en un archivo
from datetime import datetime

now = datetime.now()
nombre_results_test = 'results_test.csv'

results_test.to_csv(nombre_results_test, mode='w')

### Prueba del modelo con un ejemplo real

In [11]:
test_correo = "no tengo dinero para viajar"

predictions, raw_outputs = model.predict([test_correo])

print(etiquetas[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Boleto Universitario


# Referencias
- https://towardsdatascience.com/bert-text-classification-in-a-different-language-6af54930f9cb
- https://medium.com/dair-ai/beto-spanish-bert-420e4860d2c6